In [ ]:
import os, cv2, numpy as np, pandas as pd, matplotlib.pyplot as plt, tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K
from datetime import datetime

In [ ]:
# ───────── SETTINGS ─────────
IMG_HEIGHT, IMG_WIDTH = 50, 200
DETECT_SIZE = 224
CHARS = "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
CHAR_TO_IDX = {c: i + 1 for i, c in enumerate(CHARS)}
IDX_TO_CHAR = {0: ''}
IDX_TO_CHAR.update({i + 1: c for i, c in enumerate(CHARS)})

In [ ]:
TEST_DIR = r"C:\Users\tejas\Downloads\archive (1)\test\test\test"
TS = datetime.now().strftime("%Y%m%d_%H%M%S")
SUBMIT_CSV = f"SampleSubmission_{TS}.csv"

In [ ]:
# Debugging toggles
N_DEBUG = 10
VISUALIZE = True
FALLBACK_CROP = True
STRICT_CROP_REGION = False   # <- Try this if detector fails consistently

In [ ]:
# ───────── LOAD MODELS ─────────
detector = load_model("license_plate_detector.keras")
ocr_model = load_model("predict_crnn_model.keras")

In [ ]:
# ───────── HELPERS ─────────
def decode_ctc(preds, greedy=False):
    dec, _ = K.ctc_decode(
        preds,
        input_length=np.ones(preds.shape[0]) * preds.shape[1],
        greedy=greedy,
        beam_width=10,
    )
    return ["".join(IDX_TO_CHAR.get(i, '') for i in seq if i > 0) for seq in dec[0].numpy()]

In [ ]:
def show(image, title=""):
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis("off")
    plt.show()

In [ ]:
# ───────── MAIN LOOP ─────────
print("Processing test images …")
files = sorted(
    [f for f in os.listdir(TEST_DIR) if f.lower().endswith(('.jpg', '.png'))],
    key=lambda x: int("".join(filter(str.isdigit, x)) or 0)
)
print("Total images:", len(files))

In [ ]:
results = []
warn_ocr = 0
warn_detect = 0

In [ ]:
for idx, fname in enumerate(files):
    path = os.path.join(TEST_DIR, fname)
    img = cv2.imread(path)
    if img is None:
        print(f"[WARN] Could not read {fname}")
        results.append([fname, ""])
        continue

    h, w = img.shape[:2]

    # DETECT BOX
    inp = cv2.resize(img, (DETECT_SIZE, DETECT_SIZE)).astype("float32") / 255.0
    y1, x1, y2, x2 = detector.predict(inp[None])[0]

    if idx < N_DEBUG:
        print(f"[DEBUG] {fname} | img={h}x{w} | box={[round(v, 3) for v in (y1,x1,y2,x2)]}")

    # scale bbox
    ymin, ymax = int(y1 * h), int(y2 * h)
    xmin, xmax = int(x1 * w), int(x2 * w)

    crop = img[ymin:ymax, xmin:xmax]

    # Fallback: use center crop or hardcoded region if detector fails
    if crop.size == 0:
        warn_detect += 1
        if STRICT_CROP_REGION:
            ymin, ymax = int(h * 0.6), int(h * 0.75)
            xmin, xmax = int(w * 0.3), int(w * 0.7)
        else:
            cy, cx = h // 2, w // 2
            ymin, ymax = cy - 25, cy + 25
            xmin, xmax = cx - 100, cx + 100
        crop = img[max(0, ymin):min(h, ymax), max(0, xmin):min(w, xmax)]

    if crop.size == 0:
        print(f"[WARN] No crop for {fname}")
        results.append([fname, ""])
        continue

    # OCR
    gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, (IMG_WIDTH, IMG_HEIGHT)).astype("float32") / 255.0
    preds = ocr_model.predict(gray[None, ..., None])
    text = decode_ctc(preds)[0] or decode_ctc(preds, greedy=True)[0]

    if text == "":
        warn_ocr += 1

    results.append([fname, text])

    # VISUALIZATION
    if VISUALIZE and idx < 5:
        vis = img.copy()
        cv2.rectangle(vis, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
        cv2.putText(vis, text or "(empty)", (xmin, max(0, ymin - 10)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        show(vis, f"{fname} → {text}")

In [ ]:
print(f"\n Detection fallback used on {warn_detect} / {len(files)}")
print(f"  OCR failed (empty) on     {warn_ocr} / {len(files)}")

In [ ]:
# ───────── SUBMISSION CSV ─────────
rows = []
print("\n[CSV ROWS]")
for fname, txt in results:
    digits = {int(d) for d in txt if d.isdigit()}
    row = [os.path.splitext(fname)[0] + "_1"] + [1 if d in digits else 0 for d in range(10)]
    print(f"{fname:<12}  text='{txt}'  digits={sorted(digits)}  →  {row}")
    rows.append(row)

In [ ]:
cols = ["id"] + [str(i) for i in range(10)]
pd.DataFrame(rows, columns=cols).to_csv(SUBMIT_CSV, index=False)
print("\n Submission saved to:", SUBMIT_CSV)